In [ ]:
!pip install langchain langchain-community langchain-core langchain-groq faiss-cpu sentence-transformers docling

In [ ]:
import os
import warnings
from docling.document_converter import DocumentConverter
from pathlib import Path
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [ ]:
# Document conversion
def load_and_convert_document(file_path):
    converter = DocumentConverter()
    result = converter.convert(file_path)
    return result.document.export_to_markdown()

source = "/content/amazon-10-q-q3-2024.pdf"
markdown_content = load_and_convert_document(source)

In [ ]:
# Splitting markdown content into chunks
def get_markdown_splits(markdown_content):
    headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3")]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers=False)
    return markdown_splitter.split_text(markdown_content)
chunks = get_markdown_splits(markdown_content)

In [ ]:
# Embedding and vector store setup
def setup_vector_store(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    single_vector = embeddings.embed_query("this is some text data")
    index = faiss.IndexFlatL2(len(single_vector))
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={}
    )
    vector_store.add_documents(documents=chunks)
    return vector_store

vector_store = setup_vector_store(chunks)

# Setup retriever
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 3})

In [ ]:
# Formatting documents for RAG
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [ ]:
# Setting up the RAG chain
def create_rag_chain(retriever):
    prompt = """
        You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
        If you don't know the answer, just say that you don't know.
        Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
        Question: {question}
        Context: {context}
        Answer:
    """
    model = ChatGroq(model_name="llama3-8b-8192", api_key = "")  # ✅ Choose a Groq-supported model
    prompt_template = ChatPromptTemplate.from_template(prompt)
    return (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt_template
        | model
        | StrOutputParser()
    )

In [ ]:
# Create RAG chain
rag_chain = create_rag_chain(retriever)

questions = [
    "What was Amazon’s total revenue in Q3 2024?",
    "How does the revenue in Q3 2024 compare to Q3 2023?",
    "What was the net income for Q3 2024, and how does it compare year over year?",
    "What were the earnings per share (basic and diluted) in Q3 2024?",
    "How much did Amazon earn from product vs. service sales?",
    "What were the main operating expense categories and their values in Q3 2024?",
    "What was Amazon’s operating income in Q3 2024?",
    ]


for question in questions:
      print(f"Question: {question}")
      for chunk in rag_chain.stream(question):
          print(chunk, end="", flush=True)
      print("\n" + "-" * 50 + "\n")

In [ ]:
# Main execution logic
if __name__ == "__main__":
    # Load document
    source = "/content/amazon-10-q-q3-2024.pdf"
    markdown_content = load_and_convert_document(source)
    chunks = get_markdown_splits(markdown_content)

    # Create vector store
    vector_store = setup_vector_store(chunks)

    # Setup retriever
    retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 3})

    # Create RAG chain
    rag_chain = create_rag_chain(retriever)

    # Questions for retrieval
    # Questions for retrieval (Amazon-specific)
    questions = [
    # Financial Performance
    "What was Amazon’s total revenue in Q3 2024?",
    "How does the revenue in Q3 2024 compare to Q3 2023?",
    "What was the net income for Q3 2024, and how does it compare year over year?",
    "What were the earnings per share (basic and diluted) in Q3 2024?",
    "How much did Amazon earn from product vs. service sales?",
    "What were the main operating expense categories and their values in Q3 2024?",
    "What was Amazon’s operating income in Q3 2024?",
    ]



    # Answer questions
    for question in questions:
        print(f"Question: {question}")
        for chunk in rag_chain.stream(question):
            print(chunk, end="", flush=True)
        print("\n" + "-" * 50 + "\n")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
import gradio as gr

# Global variable to store vector store and RAG chain after file upload
vector_store = None
rag_chain = None

def upload_and_process(file):
    global vector_store, rag_chain
    file_path = file.name
    markdown_content = load_and_convert_document(file_path)
    chunks = get_markdown_splits(markdown_content)
    vector_store = setup_vector_store(chunks)
    retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 3})
    rag_chain = create_rag_chain(retriever)
    return "✅ Document processed and RAG chain created. You can now ask questions."

def ask_question(question):
    if rag_chain is None:
        return "❌ Please upload and process a document first."
    response = ""
    for chunk in rag_chain.stream(question):
        response += chunk
    return response

# Launch Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## 📊 Amazon 10-Q RAG-Based QA System using Groq + FAISS")

    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="📁 Upload Amazon 10-Q PDF", file_types=[".pdf"])
            upload_btn = gr.Button("🔄 Upload & Process")
            upload_output = gr.Textbox(label="Processing Status", lines=2)

        with gr.Column():
            question_input = gr.Textbox(label="❓ Ask a Question", placeholder="e.g. What was Amazon’s total revenue in Q3 2024?")
            ask_btn = gr.Button("🔍 Get Answer")
            answer_output = gr.Textbox(label="📢 Answer", lines=10)

    upload_btn.click(fn=upload_and_process, inputs=[file_input], outputs=[upload_output])
    ask_btn.click(fn=ask_question, inputs=[question_input], outputs=[answer_output])

demo.launch()


In [ ]:
# Main execution logic
if __name__ == "__main__":
    # Load document
    source = "/content/amazon-10-q-q3-2024.pdf"
    markdown_content = load_and_convert_document(source)
    chunks = get_markdown_splits(markdown_content)

    # Create vector store
    vector_store = setup_vector_store(chunks)

    # Setup retriever
    retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 3})

    # Create RAG chain
    rag_chain = create_rag_chain(retriever)

    # Questions for retrieval
    # Questions for retrieval (Amazon-specific)
    questions = [
    # Financial Performance
    "What was Amazon’s total revenue in Q3 2024?",
    "How does the revenue in Q3 2024 compare to Q3 2023?",
    "What was the net income for Q3 2024, and how does it compare year over year?",
    "What were the earnings per share (basic and diluted) in Q3 2024?",
    "How much did Amazon earn from product vs. service sales?",
    "What were the main operating expense categories and their values in Q3 2024?",
    "What was Amazon’s operating income in Q3 2024?",

    # Segment Analysis
    "What was the revenue and operating income for Amazon Web Services (AWS) in Q3 2024?",
    "How did the North America and International segments perform in terms of revenue and profit?",
    "Which segment contributed the most to Amazon’s operating income?",

    # Cash Flow & Capital Expenditures
    "How much cash did Amazon generate from operating activities?",
    "What were the major uses of cash in investing and financing activities?",
    "What were Amazon’s capital expenditures in Q3 2024?",

    # Debt and Liabilities
    "What is Amazon’s total long-term debt as of September 30, 2024?",
    "What are the upcoming debt maturities and interest rates for Amazon's outstanding bonds?",

    # Stock & Shareholder Equity
    "What were the changes in stockholders’ equity this quarter?",
    "Did Amazon repurchase any stock in Q3 2024?",

    # Other Financial Indicators
    "What are the values of accounts receivable, inventories, and marketable securities as of September 30, 2024?",
    "What were Amazon’s total assets and liabilities?",

    # Legal and Regulatory
    "What major legal proceedings or lawsuits was Amazon involved in during Q3 2024?",
    "What was the outcome of the Kove IO patent case?",
    "Are there any significant antitrust or consumer protection investigations ongoing?",

    # Taxation and Accounting
    "What is Amazon’s effective tax rate for Q3 2024?",
    "What discrete tax items impacted Amazon’s tax expense?",
    "Are there any significant tax disputes or contingencies noted?",

    # Revenue Breakdown
    "What was the revenue breakdown by business line (Online stores, AWS, advertising, etc.)?",
    "How did subscription services and advertising revenue perform compared to last year?"
]



    # Answer questions
    for question in questions:
        print(f"Question: {question}")
        for chunk in rag_chain.stream(question):
            print(chunk, end="", flush=True)
        print("\n" + "-" * 50 + "\n")
